In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import azureml.core

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

In [ ]:
import json
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)


In [ ]:
import tensorflow
tensorflow.__version__

In [ ]:
import uuid

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()

In [ ]:
from azureml.core import Run, Experiment, Workspace, Model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.webservice import Webservice, AciWebservice
import json

In [ ]:
with open('states/model_details.json', 'r') as f:
    model_details = json.load(f)
model = Model.deserialize(workspace=ws, model_payload=model_details['model'])


In [ ]:

env = Environment('tveer-deployment-env')
cd = CondaDependencies.create(
    pip_packages=['azureml-defaults','numpy', 'tensorflow']
)
env.python.conda_dependencies = cd
env.register(workspace = ws)

In [ ]:
inference_config = InferenceConfig(entry_script='ai-training/score.py', environment=env)
aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=1, 
    memory_gb=1, 
    tags={"data": "tveer",  "method" : "keras"}, 
    description='Classify images with tveer'
)

service_name = 'tveerdetection-svc-' + str(uuid.uuid4())[:4]
service = Model.deploy(workspace=ws, 
                    name=service_name, 
                    models=[model], 
                    inference_config=inference_config, 
                    deployment_config=aciconfig)



In [ ]:
service.wait_for_deployment(show_output=True)

with open("states/service_details.json", "w") as service_details:
    json.dump(service.serialize(), service_details)

## Test deployment

In [ ]:
from azureml.core import Dataset

In [ ]:
train_test_dataset = Dataset.get_by_name(ws, name='train_test_dataset')

In [ ]:
test_data_path = os.path.join(os.getcwd(), 'data/tveer/test_data')
os.makedirs(test_data_path, exist_ok=True)

In [ ]:
mnt_ctx_test_data = train_test_dataset.mount(test_data_path)

mnt_ctx_test_data.start()


In [ ]:
X_test_conv = np.load(os.path.join(test_data_path, 'X_test_conv.npy'))
X_test_values = np.load(os.path.join(test_data_path, 'X_test_values.npy'))
y_test = np.load(os.path.join(test_data_path, 'y_test.npy'))

In [ ]:
labels = np.load(os.path.join(test_data_path, 'component_names.npy')).tolist()

In [ ]:
%%time
index = np.random.randint(0, len(y_test))
data = (X_test_conv[index], X_test_values[index])
plt.imshow(X_test_conv[index])
plt.title("Name: {}, size: {}".format(labels[np.argmax(y_test[index])], X_test_values[index]))

dumped = json.dumps({
    "conv_data": np.expand_dims(X_test_conv[index], 0),
    "values": np.expand_dims(np.asarray(X_test_values[index]), 0)
}, cls=NumpyEncoder)

response = service.run(dumped)
y_pred = np.argmax(response['result'], axis=1)[0]
print(f"We think this object is a: {labels[y_pred]}")